# AWS Virtual Private Cloud

A VPC is a logically isolated networking environment for your AWS resources. In this notebook we'll create a VPC, and partition it into public and private subnets, showing how we can use subnets as security containers or boundaries.

## Creating a VPC

In [7]:
import boto3

client = boto3.client('ec2')
vpc_response = client.create_vpc(
    CidrBlock='10.0.0.0/16'
)

print vpc_response

{'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': 'a327bd93-af67-4f49-a4f8-d90a034f5302', 'HTTPHeaders': {'transfer-encoding': 'chunked', 'vary': 'Accept-Encoding', 'server': 'AmazonEC2', 'content-type': 'text/xml;charset=UTF-8', 'date': 'Sun, 19 Feb 2017 15:28:23 GMT'}}, u'Vpc': {u'VpcId': 'vpc-ed9feb8b', u'InstanceTenancy': 'default', u'Tags': [], u'Ipv6CidrBlockAssociationSet': [], u'State': 'pending', u'DhcpOptionsId': 'dopt-363b2f54', u'CidrBlock': '10.0.0.0/16', u'IsDefault': False}}


## Subnets

Now that we have a VPC, we can create subnets. In contrast to a VPC, which spans all the availability zones in
a region, a subnet is specific to an availability zone.

So the first thing to figure out is what are the availability zones in the region.

In [11]:
response = client.describe_availability_zones()
availability_zones = response['AvailabilityZones']

for az in availability_zones:
    print az

{u'State': 'available', u'RegionName': 'us-east-1', u'Messages': [], u'ZoneName': 'us-east-1a'}
{u'State': 'available', u'RegionName': 'us-east-1', u'Messages': [], u'ZoneName': 'us-east-1b'}
{u'State': 'available', u'RegionName': 'us-east-1', u'Messages': [], u'ZoneName': 'us-east-1c'}
{u'State': 'available', u'RegionName': 'us-east-1', u'Messages': [], u'ZoneName': 'us-east-1d'}
{u'State': 'available', u'RegionName': 'us-east-1', u'Messages': [], u'ZoneName': 'us-east-1e'}


One we know the zones, we can figure out if we can create a subnet in the zone by doing a dry run

## Clean Up

In [12]:
# Delete the VPC
response = client.delete_vpc(
    VpcId = vpc_response['Vpc']['VpcId']
)

print response

{'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': '6a837279-b8fb-4e88-b51c-6862e5d4f3ce', 'HTTPHeaders': {'transfer-encoding': 'chunked', 'vary': 'Accept-Encoding', 'server': 'AmazonEC2', 'content-type': 'text/xml;charset=UTF-8', 'date': 'Sun, 19 Feb 2017 15:36:31 GMT'}}}
